In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
from skimage import color
from os import listdir
from os.path import isfile, join
import math

In [ ]:
ipfolder = "C:\\DragonBallZ\\Summer2018\\Video_Image_Processing_GANS\\sharedVBOX\\bedroomip"
opfolder = "C:\\DragonBallZ\\Summer2018\\Video_Image_Processing_GANS\\sharedVBOX\\bedroomop"

In [ ]:
def numeric_chars2(x):
    y = x[5:-4]
    return(int(y))

In [ ]:
dct = {}

In [ ]:
niter = 100

In [ ]:
ipfiles = [f for f in os.listdir(ipfolder)]
ipfiles

In [ ]:
file_list = os.listdir(opfolder)
l1 = sorted(file_list, key = numeric_chars2)
print(l1)

In [ ]:
def kf(a,b):
    I1 = np.asarray(a.convert('L'))
    I2 = np.asarray(b.convert('L'))
    i = I1 - I2
    #revsigma=1.0/100000000 #10^8
    revsigma=1.0/10000000 #10^7
    val = np.exp(-0.5*revsigma*np.linalg.norm(i)**2)
    return val

In [ ]:
def getRandomFromOutput(dct,img):
    A1,A2 = np.random.choice(list(dct.keys()),2,replace=False)
    xg = img.crop(dct[A1])
    xgprime = img.crop(dct[A2])
    return xg,xgprime   

In [ ]:
def computeKernel(xr,xrprime,xg,xgprime):
    exp1 = kf(xr,xrprime)
    exp2 = -2 * kf(xr,xg)
    exp3 = kf(xg,xgprime)
    exp = (exp1 + exp2 + exp3)**0.5
    print("exp ",exp)
    if math.isnan(exp):
        exp = 0 
    return exp

In [ ]:
def getRandomFromInput():
    v1,v2 = np.random.choice(ipfiles,2,replace=False)
    print("IP1 ",v1)
    print("IP2 ",v2)
    inpi1 = os.path.join(ipfolder,v1)
    inpi2 = os.path.join(ipfolder,v2)
    xr = Image.open(inpi1)
    xrprime = Image.open(inpi2)
    print("xr size ",xr.size)
    print("xrprime size ",xrprime.size)
    return xr,xrprime

In [ ]:
xr,xrprime = getRandomFromInput()

In [ ]:
def populatedct(size):
    listSize = [(size*i)/8 for i in range(0,8)]
    startIndex = 1
    for col in listSize:
        for row in listSize:
            area = (row,col,row+(size/8),col+(size/8))
            dct[startIndex] = area
            startIndex+=1
    print(dct)
    return dct
    

In [ ]:
# Reshapes every incoming image to 64*64 
def reshapeImg(img):
    new_width,new_height = 64,64
    img = img.resize((new_width, new_height), Image.ANTIALIAS)
    return img
    

In [ ]:
new_height,new_width = 0,0
kernelList = []
emptydic = True 
for op in l1:
    xr,xrprime = getRandomFromInput()
    oppath = os.path.join(opfolder,op)
    img = Image.open(oppath)
    size = img.size[0]
    print(size)
    if emptydic :
        dct = populatedct(size)
        emptydic = False
    xr = reshapeImg(xr)
    xrprime = reshapeImg(xrprime)
    kernelAvglist = []
    for i in range(niter):
        xg,xgprime = getRandomFromOutput(dct,img)
        xg = reshapeImg(xg)
        xgprime = reshapeImg(xgprime)
        kernelval = computeKernel(xr,xrprime,xg,xgprime)
        kernelAvglist.append(kernelval) 
    kernelList.append(sum(kernelAvglist)/len(kernelAvglist))
plt.plot(kernelList,'.-')
plt.show()

In [ ]:
plt.plot(kernelList,'.')
plt.show()